In [1]:
#@title 0. 로컬(colab) 한글 환경 설정
# 그래프에 한글을 표기하기 위한 설정 (폰트 설치 + ...)
!sudo apt-get update -qq
!sudo apt-get install -y fonts-nanum fonts-nanum-coding
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib
!rm -rf /tmp/.cache/matplotlib

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum fonts-nanum-coding
0 upgraded, 2 newly installed, 0 to remove and 57 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum-coding all 2.5-3 [4,988 B]
Fetched 10.3 MB in 2s (5,186 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to

In [2]:
import getpass
svcKey = getpass.getpass("svcKey 입력: ")

svcKey 입력: ··········


In [11]:
#@title 1. API 데이터 수집 테스트
import requests

url = 'http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev'

params = {'serviceKey' : svcKey, 'pageNo' : '1', 'numOfRows' : '10'}

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="utf-8" standalone="yes"?><response><header><resultCode>000</resultCode><resultMsg>OK</resultMsg></header><body><items/><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>0</totalCount></body></response>'


In [14]:
import xml.etree.ElementTree as ET

print(ET.tostring(root, encoding='unicode'))


<response><header><resultCode>000</resultCode><resultMsg>OK</resultMsg></header><body><items /><numOfRows>9999</numOfRows><pageNo>1</pageNo><totalCount>0</totalCount></body></response>


In [21]:
#@title 2. 데이터 가져오기 위한 준비_1
# zcode에 대해, pageNo, numOfRows 를 지정해서 데이터를 가져오는 API 밖에 정의되어 있지 않네...
import xml.etree.ElementTree as ET

# Update params for the initial request
params['pageNo'] = '1'
params['numOfRows'] = '9999'
params['LAWD_CD'] = '11680'
params['DEAL_YMD'] = '202301'

# Make the API request
response = requests.get(url, params=params)

# Parse the XML response
root = ET.fromstring(response.content)

# Extract totalCount
total_count = int(root.find('body').find('totalCount').text)

# Extract all item elements
all_items = root.find('body').find('items').findall('item')

print(f"Total count: {total_count}")
print(f"Number of items in the first page (up to 9999): {len(all_items)}")

Total count: 108
Number of items in the first page (up to 9999): 108


In [20]:
import pandas as pd
import math
import requests
import xml.etree.ElementTree as ET

url = 'http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev'
data = []

numOfRows = 9999
pageNo = 1

while True:
    params = {
    'serviceKey': svcKey,
    'LAWD_CD': '11680',
    'DEAL_YMD': '202301',
    'pageNo': str(pageNo),
    'numOfRows': str(numOfRows)
    }

    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)

    items = root.find('body').find('items').findall('item')

    # 데이터가 없으면 종료
    if not items:
        break

    for item in items:
        row = {child.tag: child.text for child in item}
        data.append(row)

    print(f"{pageNo} 페이지 수집 완료 ({len(items)}개)")
    pageNo += 1

df = pd.DataFrame(data)
print(df.shape)
df.head()

1 페이지 수집 완료 (108개)
(108, 32)


,aptDong,aptNm,aptSeq,bonbun,bubun,buildYear,buyerGbn,cdealDay,cdealType,dealAmount,...,roadNmBonbun,roadNmBubun,roadNmCd,roadNmSeq,roadNmSggCd,roadNmbCd,sggCd,slerGbn,umdCd,umdNm
0,109,목련타운,11680-301,0716,0000,1993,,,,"175,000",...,00015,00000,4166071,01,11680,0,11680,,11400,일원동
1,301,삼익대청아파트,11680-279,0012,0000,1992,,,,"97,000",...,00021,00000,4166040,01,11680,0,11680,,10300,개포동
2,101,푸른마을아파트101동~111동,11680-308,0719,0000,1994,,,,"123,500",...,00025,00000,4166705,00,11680,,11680,,11400,일원동
3,215,성원대치2단지아파트,11680-280,0012,0000,1992,,,,"85,500",...,00009,00000,4166040,01,11680,0,11680,,10300,개포동
4,비,타워팰리스1,11680-364,0467,0000,2002,,,,"330,000",...,00056,00000,4166604,01,11680,0,11680,,11800,도곡동


In [22]:
#@title 3. 데이터 가져오기 위한 준비_2
# numOfRows를 최대치로 했는데 전부 못 가져왔으니, 나누어서 데이터를 가져와야겠네. 몇번에 나누어서 가져오면 좋을까?
import math

num_of_rows = int(params['numOfRows'])
total_pages = math.ceil(total_count / num_of_rows)

print(f"Total pages to retrieve all data: {total_pages}")

Total pages to retrieve all data: 1


In [23]:
#@title 5. 데이터 수집 확인
import pandas as pd

# Prepare a list to hold dictionaries of item data
data = []

# Iterate through each item element and extract relevant information
for item in all_items:
    item_data = {}
    for child in item:
        item_data[child.tag] = child.text
    data.append(item_data)

# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df[:5]

,aptDong,aptNm,aptSeq,bonbun,bubun,buildYear,buyerGbn,cdealDay,cdealType,dealAmount,...,roadNmBonbun,roadNmBubun,roadNmCd,roadNmSeq,roadNmSggCd,roadNmbCd,sggCd,slerGbn,umdCd,umdNm
0,109,목련타운,11680-301,0716,0000,1993,,,,"175,000",...,00015,00000,4166071,01,11680,0,11680,,11400,일원동
1,301,삼익대청아파트,11680-279,0012,0000,1992,,,,"97,000",...,00021,00000,4166040,01,11680,0,11680,,10300,개포동
2,101,푸른마을아파트101동~111동,11680-308,0719,0000,1994,,,,"123,500",...,00025,00000,4166705,00,11680,,11680,,11400,일원동
3,215,성원대치2단지아파트,11680-280,0012,0000,1992,,,,"85,500",...,00009,00000,4166040,01,11680,0,11680,,10300,개포동
4,비,타워팰리스1,11680-364,0467,0000,2002,,,,"330,000",...,00056,00000,4166604,01,11680,0,11680,,11800,도곡동


In [24]:
#@title 1. 데이터 기본 정보 확인
# 데이터 샘플 확인
# print(df.head())
# print(df.tail())

# 전체 컬럼과 타입, 결측치 확인
# print(df.info())

# 행/열 개수
# print(df.shape)
print(df.count())

# 각 컬럼의 데이터 타입
# print(df.dtypes)

# 데이터 구조 파악
# print(df.columns)
# print(df.index)
# print(df.values)

# # 특정 조건의 데이터 샘플 확인; dtype이 object인 컬럼의 값을 정렬해서 출력하고 갯수를 표시
# columns_object_dtype = df.columns[df.dtypes=='object']
# # print(columns_object_dtype)
# for col in columns_object_dtype:
#     # print (sorted(df[col].unique()))
#     # unique_values = sorted(df[col].unique())
#     unique_values = df[col].unique()
#     print(f'{col}: {len(unique_values)}개')
#     print(unique_values, '\n')

# colab의 cell 안에서 모든 컬럼을 다 표시하기 위한 설정
# pd.set_option('display.max_columns', None)

aptDong             108
aptNm               108
aptSeq              108
bonbun              108
bubun               108
buildYear           108
buyerGbn            108
cdealDay            108
cdealType           108
dealAmount          108
dealDay             108
dealMonth           108
dealYear            108
dealingGbn          108
estateAgentSggNm    108
excluUseAr          108
floor               108
jibun               108
landCd              108
landLeaseholdGbn    108
rgstDate            108
roadNm              108
roadNmBonbun        108
roadNmBubun         108
roadNmCd            108
roadNmSeq           108
roadNmSggCd         108
roadNmbCd           108
sggCd               108
slerGbn             108
umdCd               108
umdNm               108
dtype: int64
